Imports

In [1]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification,GPT2LMHeadModel, get_scheduler, AutoModelForCausalLM
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
import evaluate


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
ERROR: /home/arjun/NewPytorchEnv/bin/python3.10: undefined symbol: cudaRuntimeGetVersion
CUDA SETUP: libcudart.so path is None
CUDA SETUP: Is seems that your cuda installation is not in your path. See https://github.com/TimDettmers/bitsandbytes/issues/85 for more information.
CUDA SETUP: CUDA version lower than 11 are currently not supported for LLM.int8(). You will be only to use 8-bit optimizer

/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('0'), PosixPath('1')}
  warn(msg)
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('local/PC'), PosixPath('@/tmp/.ICE-unix/1592,unix/PC')}
  warn(msg)
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path wer

Dataset preparation and tokenising

In [2]:
dataset = load_dataset("tatsu-lab/alpaca")

Using custom data configuration tatsu-lab--alpaca-2b32f0433506ef5f
Found cached dataset parquet (/home/arjun/.cache/huggingface/datasets/tatsu-lab___parquet/tatsu-lab--alpaca-2b32f0433506ef5f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
dataset['train'].features

{'instruction': Value(dtype='string', id=None),
 'input': Value(dtype='string', id=None),
 'output': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None)}

In [4]:
dataset = dataset['train']
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 52002
})

In [5]:
dataset[0]

{'instruction': 'Give three tips for staying healthy.',
 'input': '',
 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}

In [6]:
# Making dataset smaller for fast training
dataset = dataset.select(range(10))

In [7]:
tokenizer = AutoTokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')

new_dataset = {'input_ids':[], 'attention_mask':[]}

for example in dataset:
    input_text = example['text'].replace('\n', '').replace('###','\n')   
    encoded_data = tokenizer('<|startoftext|>' + input_text + '<|endoftext|>',truncation=True, max_length=768, padding="max_length")
    new_dataset['input_ids'].append(encoded_data['input_ids'])
    new_dataset['attention_mask'].append(encoded_data['attention_mask'])

new_dataset = Dataset.from_dict(new_dataset)
new_dataset.set_format("torch")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
new_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 10
})

DataLoader

In [9]:
dataloader = DataLoader(new_dataset,shuffle=True, batch_size=8)

Loading model

In [10]:
model = GPT2LMHeadModel.from_pretrained("gpt2")

Optimiser and scheduler

In [11]:
num_epochs = 50

optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = num_epochs * len(dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [12]:
model.resize_token_embeddings(len(tokenizer))

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# device = torch.device("cpu")
model.to(device)
print(device)


cuda


In [13]:
progress_bar = tqdm(range(num_training_steps-1),desc='Training', unit='steps')
model.train()   # Some layers behave differently to training and inference. This sets all those 
                 # layers into training mode
for epoch in range(num_epochs):
    total_train_loss=0
    for batch in dataloader:
        # print(batch)
        batch_data = batch['input_ids'].to(device)
        attention = batch['attention_mask'].to(device)
        # batch_data = batch_data.flatten()  # Flatten the tensor to 1-dimensional
        # outputs = model.generate(batch_data.unsqueeze(0))
        model.zero_grad()  
        outputs = model(  batch_data,
                          labels=batch_data, 
                          attention_mask = attention,
                          token_type_ids=None
                        )

        # metric.add_batch(predictions=predictions, references=batch["input_ids"])
        loss = outputs[0]     # compute loss 
        batch_loss = loss.item()
        total_train_loss += batch_loss
        loss.backward()          # computes gradients
        optimizer.step()         # optimises
        lr_scheduler.step()      # updates lr according to schedule. Improves performance
        optimizer.zero_grad()    # resets the gradients
        progress_bar.update(1)   # updates progress bar by 1
    avg_train_loss = total_train_loss / len(dataloader)
    print('Average training loss = ',avg_train_loss)


Training:   0%|          | 0/99 [00:00<?, ?steps/s]

Average training loss =  12.443091869354248
Average training loss =  8.096329927444458
Average training loss =  3.290835738182068
Average training loss =  2.036713182926178
Average training loss =  1.054311841726303
Average training loss =  0.947840541601181
Average training loss =  0.8149786591529846
Average training loss =  0.8064834177494049
Average training loss =  0.7088437527418137
Average training loss =  1.016710788011551
Average training loss =  0.7013344466686249
Average training loss =  0.8445207476615906
Average training loss =  0.7188147008419037
Average training loss =  0.7263825833797455
Average training loss =  0.5998924076557159
Average training loss =  0.6591821908950806
Average training loss =  0.5636792927980423
Average training loss =  0.7996491193771362
Average training loss =  0.5238657146692276
Average training loss =  0.5843227803707123
Average training loss =  0.5765763819217682
Average training loss =  0.531791940331459
Average training loss =  0.528712674975

In [14]:
question = 'Give three tips for staying healthy'

prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
 Instruction:{question}
 Response: """
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0).to(device)

sample_outputs = model.generate(
                                generated, 
                                do_sample=True,   
                                top_k=300,
                                max_length=200,
                                max_new_tokens=200,
                                top_p=1, 
                                num_return_sequences= 5,
                                temperature = .5
                                )

for i, sample_output in enumerate(sample_outputs):
    ans = tokenizer.decode(sample_output, skip_special_tokens=True).split('Response: ')
    print("\n\n-------------------------------------------------------------------------------------------------------------------------------------------")
    try:        print(f'<-{i+1}-> {ans[1]}')
    except:
        print(f'<-{i+1}-> ___No response___')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=200) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




-------------------------------------------------------------------------------------------------------------------------------------------
<-1->  I  task
Response:  I received a request from a client
Response:  I received a request from a  client
Response:  I received a request from a  client
Response:  I received a request from a  client
Response:  I received a request from a  client
Response  respond   response
Response:  I received a request from a  client
Response:  I received a request from a  client
Response:  I received a request from a  client
Response:  I received a request from a  client
Response:  I received a request from a  client
Response:  I received a request from a  client
Response: I received request from a  client
Response: I received a request from a  client
Response: I received an request from  client
Response: I received a request from  client
Response: I received


------------------------------------------------------------------------------------------------